In [13]:
!pip install torch

In [3]:
!pip install pytorch-tabnet


In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score


In [39]:
data =pd.read_csv('heart_failure_clinical_records_dataset.csv')

In [40]:
data=pd.DataFrame(data)
data

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


In [41]:
X = data.drop(columns=["DEATH_EVENT"])
y = data["DEATH_EVENT"]

In [42]:
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

X_train, X_valid, y_train, y_valid = train_test_split(X.values, y.values, test_size=0.2, random_state=42)

In [43]:
# tabnet initialisation
import torch
clf = TabNetClassifier(
    n_d=8,
    n_a=8,
    n_steps=3,
    gamma=1.3,
    lambda_sparse=1e-3,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type="sparsemax"
)


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [44]:
#train the model of the cardiotrain dataset
clf.fit(
    X_train=X_train,
    y_train=y_train,
    eval_set=[(X_valid, y_valid)],
    eval_name=["val"],
    eval_metric=["accuracy"],
    max_epochs=100,    #since size of dataset <10000 max epoches can be between 100 and 200 for better results
    patience=10,
    batch_size=256,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)


epoch 0  | loss: 0.74868 | val_accuracy: 0.66667 |  0:00:00s
epoch 1  | loss: 0.68003 | val_accuracy: 0.63333 |  0:00:00s
epoch 2  | loss: 0.53842 | val_accuracy: 0.61667 |  0:00:00s
epoch 3  | loss: 0.48901 | val_accuracy: 0.63333 |  0:00:00s
epoch 4  | loss: 0.42932 | val_accuracy: 0.73333 |  0:00:00s
epoch 5  | loss: 0.45412 | val_accuracy: 0.68333 |  0:00:00s
epoch 6  | loss: 0.44182 | val_accuracy: 0.71667 |  0:00:00s
epoch 7  | loss: 0.38174 | val_accuracy: 0.68333 |  0:00:00s
epoch 8  | loss: 0.38425 | val_accuracy: 0.68333 |  0:00:00s
epoch 9  | loss: 0.36477 | val_accuracy: 0.68333 |  0:00:00s
epoch 10 | loss: 0.33609 | val_accuracy: 0.7     |  0:00:00s
epoch 11 | loss: 0.31577 | val_accuracy: 0.73333 |  0:00:00s
epoch 12 | loss: 0.33781 | val_accuracy: 0.76667 |  0:00:00s
epoch 13 | loss: 0.30525 | val_accuracy: 0.75    |  0:00:00s
epoch 14 | loss: 0.29842 | val_accuracy: 0.75    |  0:00:00s
epoch 15 | loss: 0.25823 | val_accuracy: 0.71667 |  0:00:00s
epoch 16 | loss: 0.28368

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [45]:
# evaluation

# Make predictions on the validation set
y_pred = clf.predict(X_valid)

# Calculate accuracy
accuracy = accuracy_score(y_valid, y_pred)
print("Validation Accuracy:", accuracy)


Validation Accuracy: 0.7666666666666667


In [46]:
# Feature importance (interpretability)
import numpy as np

feature_importances = clf.feature_importances_
feature_names = X.columns
print("Feature Importances:")
for name, importance in zip(feature_names, feature_importances):
    print(f"{name}: {importance:.4f}")


Feature Importances:
age: 0.0927
anaemia: 0.0955
creatinine_phosphokinase: 0.0406
diabetes: 0.0539
ejection_fraction: 0.1093
high_blood_pressure: 0.0568
platelets: 0.0942
serum_creatinine: 0.0169
serum_sodium: 0.0511
sex: 0.1512
smoking: 0.0387
time: 0.1992


inistially we encoutnred issues with emplying tab net , we decided to retrain the model wioth different epoch size cause it didnt make sense that the smoking and activity facotr had little impact on wheather or not the person has cardiovasculair issues, the resilt were the same thoguh so we kept the parameters as they are

In [47]:
torch.save(clf, 'tabnet__model_heart_fail.pth')

code snippet of how import and employ the model

In [48]:
clf_loaded = torch.load('tabnet__model_heart_fail.pth')

<ipython-input-48-12e18ad1997e>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clf_loaded = torch.load('tabnet__model_heart_fail.pth')
